# 2.2

In [270]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

Classes for the data and the neural network.

In [271]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [272]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 3),
            nn.ReLU(),
            nn.Linear(3, 3),
            nn.ReLU(),
            nn.Linear(3, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

The training and evaluation loops.

In [273]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    print(f"Train Error: \n Accuracy: {(100*correct/size):>0.1f}%\n")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Perform the learning.

In [275]:
# Configure hyperparameters
learning_rate = 1e-3
batch_size = 1
epochs = 10000

# Raw training/test data.
X = torch.tensor(np.array([
	# AND: 000
	[0, 0, 0, 0, 0],
	[0, 1, 0, 0, 0],
	[1, 0, 0, 0, 0],
	[1, 1, 0, 0, 0],
	# OR: 001
	[0, 0, 0, 0, 1],
	[0, 1, 0, 0, 1],
	[1, 0, 0, 0, 1],
	[1, 1, 0, 0, 1],
	# XOR: 010
	[0, 0, 0, 1, 0],
	[0, 1, 0, 1, 0],
	[1, 0, 0, 1, 0],
	[1, 1, 0, 1, 0],
	# NAND: 011
	[0, 0, 0, 1, 1],
	[0, 1, 0, 1, 1],
	[1, 0, 0, 1, 1],
	[1, 1, 0, 1, 1],
	# NOR: 100
	[0, 0, 1, 0, 0],
	[0, 1, 1, 0, 0],
	[1, 0, 1, 0, 0],
	[1, 1, 1, 0, 0],
])).float()
y = torch.tensor(np.array([
	0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0
]))

# Wrap in PyTorch data preparation objects.
dataset = CustomDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Determine CPU or GPU.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# Initialize the model, loss function, and optimizer.
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Conduct the learning and evaluation.
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloader, model, loss_fn, optimizer)
    test_loop(dataloader, model, loss_fn)
print("Done!")

Using cpu device
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692453 

Epoch 2
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692446 

Epoch 3
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692439 

Epoch 4
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692432 

Epoch 5
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692424 

Epoch 6
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692417 

Epoch 7
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692411 

Epoch 8
-------------------------------
Train Error: 
 Accuracy: 50.0%

Test Error: 
 Accuracy: 50.0%, Avg loss: 0.